In [5]:
import numpy as np
import os

from PIL import Image

PATH = "../benchmarks_presentation/normal/"
ASSUMPTION_SIGNATURE = "a"
CUSTOM_UNSAT_HEAD = ""

In [32]:
n_assumptions = 10000
n_mucs = 5

muc_size = 10
density = (0.5 / (n_assumptions / (muc_size * 2)))
verbose = 0

mucs = []

while True:
    assignment = np.random.binomial(n=1, p=density, size=n_assumptions)
    candidate = {i + 1 for i, x in enumerate(assignment) if x == 1}
    if sum(assignment) != muc_size:
        continue

    if verbose:
        print(assignment, candidate)

    if any([candidate.issubset(muc) or muc.issubset(candidate) for muc in mucs]):
        if verbose:
            print("DENIED")
    else:
        if verbose:
            print("APPROVED")
        mucs.append(candidate)
        if len(mucs) >= n_mucs:
            break

In [35]:
print(len(mucs))
for muc in mucs:
    print(muc)

assumption_dir = f"{n_assumptions}_assumptions"
if PATH:
    assumption_dir = f"{PATH}/{assumption_dir}"
muc_dir = f"{assumption_dir}/{n_mucs}_mucs"
dirname = muc_dir

if not os.path.isdir(assumption_dir):
    os.mkdir(assumption_dir)
if not os.path.isdir(muc_dir):
    os.mkdir(muc_dir)


img_width = 1280
img_height = max(int((n_mucs / n_assumptions) * img_width), 1)
img = np.zeros(shape=(n_mucs, n_assumptions), dtype=np.uint8) + 255
for m, muc in enumerate(mucs):
    for a in muc:
        img[m, a - 1] = 0
Image.fromarray(img).resize((img_width, img_height), resample=Image.BOX).save(f"{dirname}/visualization.jpeg")

with open(f"{dirname}/encoding.lp", "w") as f:
    f.write("% ---[ AUTO GENERATED MUC BENCHMARK ]\n")
    f.write(f"% ---[ assumptions : {n_assumptions} ]\n")
    f.write(f"% ---[ mucs : {n_mucs} ]\n\n")

    f.write(f"num(1..{n_assumptions}).\n")
    f.write("{" + ASSUMPTION_SIGNATURE + "(X): num(X)}.\n\n")

    for muc in mucs:
        f.write(f"{CUSTOM_UNSAT_HEAD}:- " + ", ".join([f"{ASSUMPTION_SIGNATURE}({a})" for a in muc]) + ".\n")

with open(f"{dirname}/assumptions.lp", "w") as f:
    for a in range(n_assumptions):
        f.write(f"assume({ASSUMPTION_SIGNATURE}({a + 1})).\n")

with open(f"{dirname}/results.txt", "w") as f:
    results = dict()
    results["minimal"] = [list([f"{ASSUMPTION_SIGNATURE}({a})" for a in muc]) for muc in mucs]

    minimum = [muc for muc in mucs if len(muc) == len(min(mucs, key=lambda x: len(x)))]
    results["minimum"] = [list([f"{ASSUMPTION_SIGNATURE}({a})" for a in muc]) for muc in minimum]

    f.write(str(results).replace("'", '"'))

with open(f"{assumption_dir}/README.md", "w") as f:
    f.write(f"# Random MUC Benchmarks [{n_assumptions} Assumptions]\n\n")
    for s, subdir in enumerate(os.walk(assumption_dir)):
        if subdir[0] != assumption_dir:
            subdir_mucs = int(subdir[0].split("/")[-1].replace("_mucs", ""))
            f.write(f"## {s} : {subdir_mucs} MUCs\n\n")
            f.write(f"![]({subdir[0].split('/')[-1]}/visualization.jpeg)\n\n")

5
{9283, 676, 2981, 7525, 6926, 5872, 8753, 2258, 8849, 2292}
{256, 7622, 842, 9389, 8625, 5714, 3443, 5010, 6069, 9883}
{5474, 9859, 6535, 3881, 2572, 8141, 9042, 602, 7579, 2494}
{2144, 2180, 5477, 8479, 9096, 5610, 749, 663, 8413, 7551}
{8640, 4227, 101, 6406, 946, 4435, 4883, 181, 284, 925}
